In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [2]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../data/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../data/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../data/medications-use.csv');

In [3]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [4]:
SELECT Medicamento.NomeMedicamento,UsoMedicamento.CodMedicamento,COUNT(*) 
FROM UsoMedicamento,Medicamento
WHERE Medicamento.codigo = UsoMedicamento.CodMedicamento
GROUP BY UsoMedicamento.CodMedicamento

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [5]:
CREATE VIEW ContagemMedicamento AS
SELECT Medicamento.NomeMedicamento,UsoMedicamento.CodMedicamento,COUNT(*) NumeroUsuarios
FROM UsoMedicamento,Medicamento
WHERE Medicamento.codigo = UsoMedicamento.CodMedicamento
GROUP BY UsoMedicamento.CodMedicamento

In [6]:
SELECT ContagemMedicamento.codmedicamento,ContagemMedicamento.NumeroUsuarios
FROM ContagemMedicamento
ORDER BY NumeroUsuarios DESC
LIMIT 1;

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [7]:
SELECT MedicamentoContagem.codmedicamento,MedicamentoContagem.NumeroUsuarios FROM
    (SELECT Medicamento.NomeMedicamento,UsoMedicamento.CodMedicamento,COUNT(*) NumeroUsuarios
    FROM UsoMedicamento,Medicamento
    WHERE Medicamento.codigo = UsoMedicamento.CodMedicamento
    GROUP BY UsoMedicamento.CodMedicamento) MedicamentoContagem
ORDER BY NumeroUsuarios DESC
LIMIT 1;

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [8]:
-- Usando o View criado anteriormente.
SELECT ContagemMedicamento.nomemedicamento,ContagemMedicamento.codmedicamento,ContagemMedicamento.NumeroUsuarios
FROM ContagemMedicamento
ORDER BY NumeroUsuarios DESC
LIMIT 1;

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [9]:
CREATE VIEW MedicamentosPorPessoa AS 
SELECT UsoMedicamento.IdPessoa,COUNT(*) NumeroMedicamentos
FROM UsoMedicamento
GROUP BY UsoMedicamento.IdPessoa

In [10]:
SELECT AVG(MedicamentosPorPessoa.NumeroMedicamentos)
FROM MedicamentosPorPessoa

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [14]:
SELECT MedicamentosPorPessoa.IdPessoa, MedicamentosPorPessoa.NumeroMedicamentos
FROM MedicamentosPorPessoa
WHERE MedicamentosPorPessoa.NumeroMedicamentos > (SELECT AVG(MedicamentosPorPessoa.NumeroMedicamentos) FROM MedicamentosPorPessoa)
ORDER BY MedicamentosPorPessoa.NumeroMedicamentos DESC

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [12]:
SELECT Pessoa.Idade, AVG(MedicamentosPorPessoa.NumeroMedicamentos) MediaMedicamentos
FROM MedicamentosPorPessoa,Pessoa
WHERE MedicamentosPorPessoa.IdPessoa = Pessoa.id
GROUP BY Pessoa.Idade